# Library Imports

In [3]:
!pip install mne

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 31.7 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import kurtosis, skew
from scipy.signal import argrelextrema, welch
from scipy.integrate import cumtrapz
import statistics
import time

import mne # to read the .edf files
import csv # to write out results
from datetime import date

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif, VarianceThreshold
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold

In [5]:
from sklearn.utils.class_weight import compute_class_weight

# Preprocessing

In [ ]:
# some pre-processing used function from pyeeg

def bin_power(X, Band, Fs):
    """Compute power in each frequency bin specified by Band from FFT result of

    Parameters
    -----------

    X: list: a 1-D real time series.
    Band; list
        boundary frequencies (in Hz) of bins. They can be unequal bins, e.g.
        [0.5,4,7,12,30] which are delta, theta, alpha and beta respectively.
        You can also use range() function of Python to generate equal bins and
        pass the generated list to this function.

        Each element of Band is a physical frequency and shall not exceed the
        Nyquist frequency, i.e., half of sampling frequency.



    Fs: integer: the sampling rate in physical frequency

    Returns
    -------

    Power: list: spectral power in each frequency bin.

    Power_ratio: list:
        spectral power in each frequency bin normalized by total power in ALL
        frequency bins.

    """

    C = np.fft.fft(X)
    C = abs(C)
    Power = np.zeros(len(Band) - 1)
    for Freq_Index in range(0, len(Band) - 1):
        Freq = float(Band[Freq_Index])
        Next_Freq = float(Band[Freq_Index + 1])
        Power[Freq_Index] = sum(
            C[int(np.floor(Freq / Fs * len(X))):
                int(np.floor(Next_Freq / Fs * len(X)))]
        )
    Power_Ratio = Power / sum(Power)
    return Power, Power_Ratio


def hfd(X, Kmax):
    """ Compute Higuchi Fractal Dimension of a time series X. kmax
     is an HFD parameter
    """
    L = []
    x = []
    N = len(X)
    for k in range(1, Kmax):
        Lk = []
        for m in range(0, k):
            Lmk = 0
            for i in range(1, int(np.floor((N - m) / k))):
                Lmk += abs(X[m + i * k] - X[m + i * k - k])
            Lmk = Lmk * (N - 1) / np.floor((N - m) / float(k)) / k
            Lk.append(Lmk)
        L.append(np.log(np.mean(Lk)))
        x.append([np.log(float(1) / k), 1])

    (p, _, _, _) = np.linalg.lstsq(x, L)
    return p[0]



def pfd(X, D=None):
    """Compute Petrosian Fractal Dimension of a time series from either two
    cases below:
        1. X, the time series of type list (default)
        2. D, the first order differential sequence of X (if D is provided,
           recommended to speed up)

    In case 1, D is computed using Numpy's difference function.

    To speed up, it is recommended to compute D before calling this function
    because D may also be used by other functions whereas computing it here
    again will slow down.
    """
    if D is None:
        D = np.diff(X)
        D = D.tolist()
    N_delta = 0  # number of sign changes in derivative of the signal
    for i in range(1, len(D)):
        if D[i] * D[i - 1] < 0:
            N_delta += 1
    n = len(X)
    return np.log10(n) / (
        np.log10(n) + np.log10(n / n + 0.4 * N_delta)
    )


def hurst(X):
    """ Compute the Hurst exponent of X. If the output H=0.5,the behavior
    of the time-series is similar to random walk. If H<0.5, the time-series
    cover less "distance" than a random walk, vice verse.

    Parameters
    ----------

    X: list: a time series

    Returns
    -------
    H: float:  Hurst exponent
    """
    X = np.array(X)
    N = X.size
    T = np.arange(1, N + 1)
    Y = np.cumsum(X)
    Ave_T = Y / T

    S_T = np.zeros(N)
    R_T = np.zeros(N)

    for i in range(N):
        S_T[i] = np.std(X[:i + 1])
        X_T = Y - T * Ave_T[i]
        R_T[i] = np.ptp(X_T[:i + 1])

    R_S = R_T / S_T
    R_S = np.log(R_S)[1:]
    n = np.log(T)[1:]
    A = np.column_stack((n, np.ones(n.size)))
    [m, c] = np.linalg.lstsq(A, R_S)[0]
    H = m
    return H


def spectral_entropy(X, Band, Fs, Power_Ratio=None):
    """Compute spectral entropy of a time series from either two cases below:
    1. X, the time series (default)
    2. Power_Ratio, a list of normalized signal power in a set of frequency
    bins defined in Band (if Power_Ratio is provided, recommended to speed up)

    In case 1, Power_Ratio is computed by bin_power() function.

    Notes
    -----
    To speed up, it is recommended to compute Power_Ratio before calling this
    function because it may also be used by other functions whereas computing
    it here again will slow down.

    Parameters
    ----------

    X: list: a 1-D real time series.

    Band: list

        boundary frequencies (in Hz) of bins. They can be unequal bins, e.g.
        [0.5,4,7,12,30] which are delta, theta, alpha and beta respectively.
        You can also use range() function of Python to generate equal bins and
        pass the generated list to this function.

        Each element of Band is a physical frequency and shall not exceed the
        Nyquist frequency, i.e., half of sampling frequency.


    Fs: integer: the sampling rate in physical frequency

    Returns
    -------

    As indicated in return line

    See Also
    --------
    bin_power: pyeeg function that computes spectral power in frequency bins

    """

    if Power_Ratio is None:
        Power, Power_Ratio = bin_power(X, Band, Fs)

    Spectral_Entropy = 0
    for i in range(0, len(Power_Ratio) - 1):
        Spectral_Entropy += Power_Ratio[i] * np.log(Power_Ratio[i])
    Spectral_Entropy /= np.log(
        len(Power_Ratio)
    )  # to save time, minus one is omitted
    return -1 * Spectral_Entropy


def hjorth(X, D=None):
    """ Compute Hjorth mobility and complexity of a time series from either two
    cases below:
        1. X, the time series of type list (default)
        2. D, a first order differential sequence of X (if D is provided,
           recommended to speed up)

    In case 1, D is computed using Numpy's Difference function.

    Notes
    -----
    To speed up, it is recommended to compute D before calling this function
    because D may also be used by other functions whereas computing it here
    again will slow down.

    Parameters
    ----------

    X: list: a time series

    D: list: first order differential sequence of a time series

    Returns
    -------

    As indicated in return line

    Hjorth mobility and complexity

    """

    if D is None:
        D = np.diff(X)
        D = D.tolist()

    D.insert(0, X[0])  # pad the first difference
    D = np.array(D)

    n = len(X)

    M2 = float(sum(D ** 2)) / n
    TP = sum(np.array(X) ** 2)
    M4 = 0
    for i in range(1, len(D)):
        M4 += (D[i] - D[i - 1]) ** 2
    M4 = M4 / n

    return np.sqrt(M2 / TP), np.sqrt(
        float(M4) * TP / M2 / M2
    )  # Hjorth Mobility and Complexity

In [ ]:
# read the edf and print stuff first
# def eeg_visualize(file, start_time, end_time):
def eeg_visualize(file):
    raw = mne.io.read_raw_edf(file)
    n = 2

    # MNE-Python's interactive data browser to get a better visualization
    raw.plot()

    # select a time frame
    start, stop = raw.time_as_index([100, 115])  # 100 s to 115 s data segment
    temp, times = raw[:, start:stop]
    fig, axs = plt.subplots(n)
    fig.suptitle('Patient EEG')
    plt.xlabel('time (s)')
    plt.ylabel('MEG data (T)')
    for i in range(n):
        axs[i].plot(times, temp[i].T)
    plt.show()

In [ ]:
# feature extracting process
def eeg_features(data):
    data = np.asarray(data)
    res  = np.zeros([22])
    Kmax = 5
    # M    = 10
    # R    = 0.3
    Band = [1,5,10,15,20,25]
    Fs   = 256
    power, power_ratio = bin_power(data, Band, Fs)
    f, P = welch(data, fs=Fs, window='hann', noverlap=0, nfft=int(256.))       # Signal power spectrum
    area_freq = cumtrapz(P, f, initial=0)
    res[0] = np.sqrt(np.sum(np.power(data, 2)) / data.shape[0])                   # amplitude RMS
    res[1] = statistics.stdev(data)**2                                            # variance
    res[2] = kurtosis(data)                                                       # kurtosis
    res[3] = skew(data)                                                           # skewness
    res[4] = max(data)                                                            # max amplitude
    res[5] = min(data)                                                            # min amplitude
    res[6] = len(argrelextrema(data, np.greater)[0])                              # number of local extrema or peaks
    res[7] = ((data[:-1] * data[1:]) < 0).sum()                                   # number of zero crossings
    res[8] = hfd(data, Kmax)                                                      # Higuchi Fractal Dimension
    res[9] = pfd(data)                                                            # Petrosian Fractal Dimension
    res[10] = hurst(data)                                                         # Hurst exponent
    res[11] = spectral_entropy(data, Band, Fs, Power_Ratio=power_ratio)           # spectral entropy (1.21s)
    res[12] = area_freq[-1]                                                       # total power
    res[13] = f[np.where(area_freq >= res[12] / 2)[0][0]]                         # median frequency
    res[14] = f[np.argmax(P)]                                                     # peak frequency
    res[15], res[16] = hjorth(data)                                               # Hjorth mobility and complexity
    res[17] = power_ratio[0]
    res[18] = power_ratio[1]
    res[19] = power_ratio[2]
    res[20] = power_ratio[3]
    res[21] = power_ratio[4]
    # res[22] = pyeeg.samp_entropy(data, M, R)             # sample entropy
    # res[23] = pyeeg.ap_entropy(data, M, R)             # approximate entropy (1.14s)
    return (res)


In [ ]:
# eeg pre-processing
def eeg_preprocessing(file, seizures, epoch_length = 10, step_size = 1, start_time = 0):
    start = time.time()

    # reading in data
    raw = mne.io.read_raw_edf(file)

    # apply filterbank
    raw = raw.load_data().filter(l_freq=0.25, h_freq=25)
    channels = raw.ch_names                                  # column names

    # Divide into epochs
    res = []
    while start_time <= max(raw.times) + 0.01 - epoch_length:  # max(raw.times) = 3600
#     while start_time <= 50 - epoch_length:  # max(raw.times) = 3600
        features = []
        start, stop = raw.time_as_index([start_time, start_time + epoch_length])
        temp = raw[:, start:stop][0]

        # start time as ID
        features.append(start_time)

        # features
        for i in range(23):
            features.extend(eeg_features(temp[i]).tolist())

        # seizure flag for y
        if file in seizures:  # if file has seizure
            for seizure in seizures[file]:
                if start_time > seizure[0] and start_time < seizure[1]:
                    features.append(1)
                elif start_time + epoch_length > seizure[0] and start_time + epoch_length < seizure[1]:
                    features.append(1)
                else:
                    features.append(0)
        else:
            features.append(0)

        res.append(features)
        start_time += step_size
        print("Section ", str(len(res)), "; start: ", start, " ; stop: ", stop)

    # formatting
    feature_names = ["rms", "variance", "kurtosis", "skewness", "max_amp", "min_amp", "n_peaks", "n_crossings",
        "hfd", "pfd", "hurst_exp", "spectral_entropy", "total_power", "median_freq", "peak_freq",
        "hjorth_mobility", "hjorth_complexity", "power_1hz", "power_5hz", "power_10hz", "power_15hz", "power_20hz"]

    column_names = ["start_time"]
    for channel in channels:
        for name in feature_names:
            column_names.append(channel + "_" + name)
    column_names.append("seizure")

    res = pd.DataFrame(res, columns=column_names)

    end = time.time()
    print("Finished preprocessing ", file, f" took {(end - start) / 60} minutes")
    return res

# PCA

In [ ]:
def pca(res, patient_num):
  # more preprocessing -- this is training specific
  X = res.loc[:, res.columns != "seizure"]
  X = X.loc[:, X.columns != "start_time"]
  X = X.loc[:, X.columns != "file ID"]

  results = {'# of features': [], 'features_idx': [], 'SVM accuracy': [], 'MLP accuracy': [], 'SVM rates': [], 'MLP rates': []}
  writeout = {}
  #for n_ft in range(1, len(X.columns)): # This one is to run through all features (~510)
  for n_ft in range(1, 3): # This one is for testing
    # Apply PCA to X values
    scaler = StandardScaler()
    scaled_X = scaler.fit_transform(X)
    pca = PCA(0.80)
    pc_X= pca.fit_transform(scaled_X)
    pc_X_df = pd.DataFrame(data = pc_X, columns = ['PC' + str(i) for i in range(1, pca.n_components_ + 1)])
    scale_factor = 10
    scaled_pc_X = (pc_X * scale_factor).astype(int)
    Y = np.asarray(res['seizure'])
    feature_names = X.columns.tolist()

    # Finding the most important features
    explained_variance_ratio = pca.explained_variance_ratio_
    loadings = pca.components_ # each row reprensents a principal component
    writeout["Explained Variance Ratio"] = ["Vector of the variance explained by each dimension", pca.explained_variance_ratio_]

    # get the absolute value of the loading
    abs_load = np.abs(loadings)
    writeout["Absolute Loadings"] = ["How much each feature influences a principal component (each row is a principal component)", abs_load]

    # get the weight of all the features from the PCA
    feature_importance = np.sum(abs_load, axis = 0)

    # sort the index of the feature importance in descending order
    # biggest features show up first
    sorted_idx = np.argsort(feature_importance)[::-1]
    writeout["Sorted IDX"] = ["List of indices of the features, sorted in descending order of importance", sorted_idx]

    # get n_ft most important features
    most_important_feature = sorted_idx[:n_ft]
    # get the new columns from the PCA
    new_columns = []
    ft_id = []
    for j in most_important_feature:
        new_columns.append(feature_names[j])
        name = str(feature_names[j])
        ft_id.append(f"{name} (ID: {j})")
    new_df = res[new_columns]
    writeout["Post-PCA Data"] = ["Data used to determine train-test splits (after some scaling/transforming)", new_df]

    # scale new_df
    n_scaler = StandardScaler()
    n_scaled_X = scaler.fit_transform(new_df)

    # quantize to int
    scaled_n_X = (n_scaled_X * scale_factor).astype(int)

    # change back to df just for visualization
    n_scaled_df = pd.DataFrame(data = scaled_n_X, columns = [feature_names[i] for i in most_important_feature])

    # used_input = scaled_pc_X
    used_input = scaled_n_X
    X_train, X_test, y_train, y_test = train_test_split(used_input, Y, test_size = 0.2, random_state=0)
    writeout["Train-Test Splits"] = ["Train-test splits used for SVM and MLP", [f"X_train: {X_train}", f"X_test: {X_test}", f"y_train: {y_train}", f"y_test: {y_test}"]]

    steps(n_ft, writeout, patient_num) # Comment out if processing steps file is unwanted (final file will only contain results from last iteration of features)

    acc_svm, rates_svm = svm(y_train, y_test, X_train, X_test)
    acc_mlp, rates_mlp = mlp(y_train, y_test, X_train, X_test)

    # results = {'# of features': [], 'features': [], 'SVM accuracy': [], 'MLP accuracy': []}
    results['# of features'].append(n_ft)
    results['features_idx'].append(ft_id)
    results['SVM accuracy'].append(acc_svm)
    results['MLP accuracy'].append(acc_mlp)

    # True Negatives + False Positives = False Negatives + True Positives = 1
    results['SVM rates'].append(rates_svm)
    results['MLP rates'].append(rates_mlp) # TODO: MLP rates seem suspicious, come back and check why

  return results

In [ ]:
def svm(y_train, y_test, X_train, X_test):
  class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
  class_weight_dict = dict(zip(np.unique(y_train), class_weights))
  model = SVC(kernel="rbf", class_weight=class_weight_dict, random_state = 0)

  kf = KFold(n_splits=5)
  accuracy, TPR, FPR = [], [], []
  for train, test in kf.split(X_train):
    # train
    model.fit(X_train[train, :], y_train[train])
    pred = model.predict(X_train[test])
    tn, fp, fn, tp = confusion_matrix(y_train[test], pred).ravel()
    accuracy.append((tp + tn)/(tn + fp + fn + tp))
    TPR.append(tp / (tp + fn))
    FPR.append(fp / (fp + tn))

  model.fit(X_train, y_train)
  pred = model.predict(X_test)
  tn, fp, fn, tp = confusion_matrix(y_test, pred).ravel()
  test_accuracy = ((tp + tn)/(tn + fp + fn + tp))
  test_TPR = (tp / (tp + fn))
  test_FPR = (fp / (fp + tn))
  test_TNR = (tn / (fp + tn))
  test_FNR = (fn / (tp + fn))

  total = len(pred)
  right = 0
  for i in range(len(pred)):
      if (pred[i] == y_test[i]):
          right += 1
  acc= (right/total)*100

  return acc, ["True Positives: " + str(test_TPR), "False Positives: " + str(test_FPR), "True Negatives: " + str(test_TNR), "False Negatives: " + str(test_FNR)]

In [ ]:
def mlp(y_train, y_test, X_train, X_test):
  model2 = MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10, 10), learning_rate='constant',
       learning_rate_init=0.001, max_iter=500, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

  kf = KFold(n_splits=5)
  accuracy2, TPR2, FPR2 = [], [], []
  for train, test in kf.split(X_train):
    # train
    model2.fit(X_train[train, :], y_train[train])
    pred2 = model2.predict(X_train[test])
    tn, fp, fn, tp = confusion_matrix(y_train[test], pred2).ravel()
    accuracy2.append((tp + tn)/(tn + fp + fn + tp))
    TPR2.append(tp / (tp + fn))
    FPR2.append(fp / (fp + tn))

  model2.fit(X_train, y_train)
  pred2 = model2.predict(X_test)
  tn2, fp2, fn2, tp2 = confusion_matrix(y_test, pred2).ravel()
  test_accuracy2 = ((tp2 + tn2)/(tn2 + fp2 + fn2 + tp2))
  test_TPR2 = (tp2 / (tp2 + fn2))
  test_FPR2 = (fp2 / (fp2 + tn2))
  #test_accuracy2
  test_TNR2 = (tn2 / (fp2 + tn2))
  test_FNR2 = (fn2 / (tp2 + fn2))

  total = len(pred2)
  right = 0
  for i in range(len(pred2)):
      if (pred2[i] == y_test[i]):
          right += 1
  acc= (right/total)*100

  return acc, ["True Positives: " + str(test_TPR2), "False Positives: " + str(test_FPR2), "True Negatives: " + str(test_TNR2), "False Negatives: " + str(test_FNR2)]

In [ ]:
# Function to write select processing steps to a txt file
def steps(num, output, patient):
  # Set name for file
  today = date.today()

  # Formatting dividers
  div1 = "-" * 20
  div2 = "*" * 50

  # Write to file
  f = open(f"ProcessSteps_Patient{patient}_{str(today.strftime('%b-%d'))}.txt", "w")
  for step in output:
    f.write(step + "\n" + output[step][0] + "\n" + div1 + "\n" + str(output[step][1]) + "\n" + div2 + "\n")
  f.close()

# Reading and Preprocessing files

In [6]:
data1 = pd.read_csv(f"preprocessed_1.csv", on_bad_lines='warn')
data6 = pd.read_csv(f"preprocessed_6.csv", on_bad_lines='warn')

In [20]:
print(np.unique(data1["seizure"], axis=0))
print(np.unique(data6["seizure"], axis=0))
print(data1["seizure"].shape)
print(data6["seizure"].shape)

[0 1]
[0]
(27453,)
(14391,)


In [ ]:
patient_nums = [1, 6]
for i in patient_nums:
  data = pd.read_csv(f"preprocessed_{i}.csv", on_bad_lines='warn')
  print(data.head())
  break
  results = pca(data, i)

  # Write key results (number of features, feature names and ID, and SVM/MLP accuracy) to CSV file
  results_table = pd.DataFrame.from_dict(results, orient='index')
  results_table.to_csv(f"patient{i}_results.csv", index=False)

  # Plot SVM and MLP accuracy as a function of the number of features used
  fig = plt.figure(f"Patient {i} Plot")
  plt.plot(results["# of features"], results["SVM accuracy"], label="SVM Accuracy")
  plt.plot(results["# of features"], results["MLP accuracy"], label="MLP Accuracy")
  plt.legend()
  plt.title("SVM vs MLP Seizure Detection Accuracy")
  plt.xlabel("Number of Features")
  plt.ylabel('Accuracy Rate (%)')
  plt.savefig(f"patient{i}_accuracies.png")

   Unnamed: 0  start_time  FP1-F7_rms  FP1-F7_variance  FP1-F7_kurtosis  \
0           0           0    0.000062     3.817093e-09         0.720216   
1           1           1    0.000054     2.960967e-09         0.624579   
2           2           2    0.000054     2.910406e-09         0.563609   
3           3           3    0.000054     2.864906e-09         0.606589   
4           4           4    0.000053     2.832845e-09         0.656068   

   FP1-F7_skewness  FP1-F7_max_amp  FP1-F7_min_amp  FP1-F7_n_peaks  \
0         0.002630        0.000204       -0.000212           160.0   
1         0.265297        0.000204       -0.000171           160.0   
2         0.295528        0.000204       -0.000171           159.0   
3         0.273589        0.000204       -0.000171           159.0   
4         0.285109        0.000204       -0.000171           158.0   

   FP1-F7_n_crossings  ...  T8-P8-1_median_freq  T8-P8-1_peak_freq  \
0               117.0  ...                  3.0           